In [23]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd

# URL du dataset WeatherBench 2 sur Google Cloud
url = "gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr"

# Ouverture lazy (ne charge rien encore en mémoire)
ds = xr.open_zarr(url, consolidated=True)

### Température

#### Cartes

In [18]:
def plot_era5_t2m(ds, date, region=(-25, 45, 35, 70), save_path=None):
    """
    Affiche la température à 2 m (T2M) depuis un dataset ERA5 pour une date donnée.

    Paramètres
    ----------
    ds : xarray.Dataset
        Dataset ERA5 contenant la variable '2m_temperature'.
    date : str
        Date au format ISO (ex. '2012-08-25T15:00:00').
    region : tuple (lon_min, lon_max, lat_min, lat_max), optionnel
        Limites géographiques de la carte (par défaut : Europe).
    save_path : str ou None, optionnel
        Chemin complet du fichier PNG à sauvegarder (ex. 'output/t2m_map.png').
        Si None, la figure est simplement affichée.

    Exemple
    -------
    >>> ds = xr.open_dataset("era5_surface.nc")
    >>> plot_era5_t2m(ds, "2012-08-25T15:00:00", region=(-10, 30, 35, 60))
    """
    
    lon_min, lon_max, lat_min, lat_max = region

    # Sélection de la date et conversion en °C
    t2m = ds["2m_temperature"].sel(time=date, method="nearest", tolerance="3H") - 273.15

    # Recentrage des longitudes sur [-180, 180]
    t2m = t2m.assign_coords(longitude=(((t2m.longitude + 180) % 360) - 180)).sortby("longitude")
    
    # --- Gestion du cas où la région traverse la discontinuité 180° ---
    if lon_min < lon_max:
        # Cas normal (région continue)
        t2m_region = t2m.sel(longitude=slice(lon_min, lon_max), latitude=slice(lat_max, lat_min))
    else:
        # Cas traversant 180° → concaténer deux morceaux
        t2m_left = t2m.sel(longitude=slice(lon_min, 180), latitude=slice(lat_max, lat_min))
        t2m_right = t2m.sel(longitude=slice(-180, lon_max), latitude=slice(lat_max, lat_min))
        t2m_region = xr.concat([t2m_left, t2m_right], dim="longitude")
    
    # Sélection de la région
    t2m_region = t2m.sel(longitude=slice(lon_min, lon_max), latitude=slice(lat_max, lat_min))

    # Projection et carte
    proj = ccrs.PlateCarree()
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection=proj)
    ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=proj)
    
    # Tracé
    im = t2m_region.plot(
        ax=ax, transform=proj, cmap="coolwarm",
        vmin=-40, vmax=40,
        cbar_kwargs={"label": "2m temperature (°C)"}
    )
    
    # Ajout des côtes, frontières et grille
    ax.add_feature(cfeature.COASTLINE.with_scale("50m"), linewidth=0.8)
    ax.add_feature(cfeature.BORDERS.with_scale("50m"), linewidth=0.6)
    gl = ax.gridlines(draw_labels=True, linestyle="--", linewidth=0.4)
    gl.right_labels = False
    gl.top_labels = False

    plt.title(f"ERA5 — 2m temperature — {str(t2m.time.values)[:19]} UTC")
    
    # Sauvegarde ou affichage
    if save_path is not None:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print(f"Carte sauvegardée sous : {save_path}")
    else:
        plt.show()
    

##### Random date

In [ ]:
date = "2012-04-25T15:00:00"

In [20]:
plot_era5_t2m(ds,date,save_path='visuels_era5/t2m_2012-04-25_europe.png')
plot_era5_t2m(ds, date, region=(-180,180,-90,90), save_path='visuels_era5/t2m_2012-04-25_monde.png')

c:\Users\arfib\anaconda3\envs\meteo_env\Lib\site-packages\xarray\core\indexes.py:631: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  flat_indexer = index.get_indexer(flat_labels, method=method, tolerance=tolerance)


Carte sauvegardée sous : visuels_era5/t2m_2012-04-25_europe.png


c:\Users\arfib\anaconda3\envs\meteo_env\Lib\site-packages\xarray\core\indexes.py:631: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  flat_indexer = index.get_indexer(flat_labels, method=method, tolerance=tolerance)


Carte sauvegardée sous : visuels_era5/t2m_2012-04-25_monde.png


##### Canicule du 11 août 2021

In [21]:
date = "2021-08-11T12:00:00"

In [22]:
plot_era5_t2m(ds,date,save_path='visuels_era5/t2m_2021-08-11_europe_canicule.png')
plot_era5_t2m(ds, date, region=(-180,180,-90,90), save_path='visuels_era5/t2m_2021-08-11_monde_canicule.png')

c:\Users\arfib\anaconda3\envs\meteo_env\Lib\site-packages\xarray\core\indexes.py:631: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  flat_indexer = index.get_indexer(flat_labels, method=method, tolerance=tolerance)


Carte sauvegardée sous : visuels_era5/t2m_2021-08-11_europe_canicule.png


c:\Users\arfib\anaconda3\envs\meteo_env\Lib\site-packages\xarray\core\indexes.py:631: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  flat_indexer = index.get_indexer(flat_labels, method=method, tolerance=tolerance)


Carte sauvegardée sous : visuels_era5/t2m_2021-08-11_monde_canicule.png


##### Vague de froid du 27 janvier au 13 février 2012

In [24]:
# Définir la période de la vague de froid
dates = pd.date_range("2012-01-27", "2012-02-13", freq="1D")

# Créer les cartes jour par jour
for date in dates:
    iso_date = date.strftime("%Y-%m-%dT12:00:00")  # milieu de journée
    output_path = f"visuels_era5/vague_froid/t2m_{date.strftime('%Y%m%d')}.png"
    plot_era5_t2m(
        ds,
        iso_date,
        save_path=output_path
    )

c:\Users\arfib\anaconda3\envs\meteo_env\Lib\site-packages\xarray\core\indexes.py:631: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  flat_indexer = index.get_indexer(flat_labels, method=method, tolerance=tolerance)


Carte sauvegardée sous : visuels_era5/vague_froid/t2m_20120127.png


c:\Users\arfib\anaconda3\envs\meteo_env\Lib\site-packages\xarray\core\indexes.py:631: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  flat_indexer = index.get_indexer(flat_labels, method=method, tolerance=tolerance)


Carte sauvegardée sous : visuels_era5/vague_froid/t2m_20120128.png


c:\Users\arfib\anaconda3\envs\meteo_env\Lib\site-packages\xarray\core\indexes.py:631: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  flat_indexer = index.get_indexer(flat_labels, method=method, tolerance=tolerance)


...

Carte sauvegardée sous : visuels_era5/vague_froid/t2m_20120211.png


c:\Users\arfib\anaconda3\envs\meteo_env\Lib\site-packages\xarray\core\indexes.py:631: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  flat_indexer = index.get_indexer(flat_labels, method=method, tolerance=tolerance)


Carte sauvegardée sous : visuels_era5/vague_froid/t2m_20120212.png


c:\Users\arfib\anaconda3\envs\meteo_env\Lib\site-packages\xarray\core\indexes.py:631: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  flat_indexer = index.get_indexer(flat_labels, method=method, tolerance=tolerance)


Carte sauvegardée sous : visuels_era5/vague_froid/t2m_20120213.png


In [33]:
import imageio.v2 as imageio
import os

def make_gif(image_folder, output_path, fps=2):
    """
    Crée un GIF animé à partir d'une série d'images PNG triées par date.

    Paramètres
    ----------
    image_folder : str
        Dossier contenant les images PNG (ex : "figures/").
    output_path : str
        Chemin du GIF de sortie (ex : "wave_froid_2012.gif").
    fps : int
        Images par seconde. Plus fps est petit, plus le GIF est lent.

    Exemple
    -------
    >>> make_gif("figures", "wave_froid_2012.gif", fps=2)
    """
    # Récupérer et trier les fichiers PNG
    files = sorted([f for f in os.listdir(image_folder) if f.endswith(".png")])
    if not files:
        print("❌ Aucun fichier PNG trouvé dans le dossier.")
        return

    # Lire les images
    images = [imageio.imread(os.path.join(image_folder, f)) for f in files]

    # Créer le GIF
    imageio.mimsave(output_path, images, fps=fps)
    print(f"✅ GIF sauvegardé : {output_path}")


In [36]:
make_gif("visuels_era5/vague_froid", "vague_froid_2012.gif",fps=2)

✅ GIF sauvegardé : vague_froid_2012.gif


#### Moyenne mensuelle sur toute la durée disponible dans plusieurs villes

In [52]:
def plot_monthly_series_t2m(ds, locations, save_path=None):
    """
    Trace une série temporelle mensuelle de la température à 2 m pour plusieurs sites.
    Chaque point correspond à la moyenne du mois.

    Paramètres
    ----------
    ds : xarray.Dataset
        Dataset ERA5 contenant '2m_temperature'.
    locations : list de dict
        Chaque dict doit contenir : {"name": "Nom", "lat": xx, "lon": yy}
        /!\ La longitude doit être exprimée dans l'intervale (-180,180)
        Exemple : [{"name":"Paris","lat":48.85,"lon":2.35}]
    save_path : str ou None
        Chemin pour sauvegarder le plot. Si None → affichage à l'écran.
    """

    plt.figure(figsize=(12, 6))
    
    t2m = ds["2m_temperature"] - 273.15 # K -> °C

    # Recentrage des longitudes sur [-180, 180]
    t2m = t2m.assign_coords(longitude=(((t2m.longitude + 180) % 360) - 180)).sortby("longitude")

    for loc in locations:
        name = loc["name"]
        lat = loc["lat"]
        lon = loc["lon"]
        
        print(f"Processing location : {name}")

        # Extraire la série temporelle du point le plus proche
        t2m_point = t2m.sel(
            latitude=lat, longitude=lon, method="nearest"
        )

        # Moyenne par mois (tous les jours d’un même mois)
        monthly_mean = t2m_point.resample(time="1M").mean()

        # Tracer
        plt.plot(monthly_mean["time"].values, monthly_mean.values, marker='o', label=name)

    plt.xlabel("Temps")
    plt.ylabel("Température moyenne à 2 m (°C)")
    plt.title("Série temporelle mensuelle de la température à 2 m")
    plt.grid(True)
    plt.legend()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        plt.close()
        print(f"✅ Image sauvegardée : {save_path}")
    else:
        plt.show()

<>:12: SyntaxWarning: invalid escape sequence '\ '
<>:12: SyntaxWarning: invalid escape sequence '\ '
C:\Users\arfib\AppData\Local\Temp\ipykernel_68800\2984955967.py:12: SyntaxWarning: invalid escape sequence '\ '
  /!\ La longitude doit être exprimée dans l'intervale (-180,180)


In [ ]:
locations = [
    {"name": "Paris", "lat": 48.85, "lon": 2.35},
    {"name": "New York", "lat": 40.7, "lon": -74.0},
    {"name": "Beijing", "lat": 39.9, "lon": 116.4}
]

# Ou sauvegarde directe
plot_monthly_series_t2m(ds, locations, save_path="monthly_series_t2m.png")


Processing location : Paris


c:\Users\arfib\anaconda3\envs\meteo_env\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


### Précipitations